In [1]:
import pandas as pd
import json
import re
import scipy.signal as signal
import sys
import datetime
import math
import os
import ast
import math , pywt , numpy as np
from functools import reduce
from scipy.stats import shapiro
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
from IPython.display import display, HTML

In [2]:
##################
####
### Data preparation
####
##################

In [3]:
Experiments =  {
        "Exp1": {
        "ETDataPath": "rawData/eventsDataWithAois.csv",
        "AnswerDataPath": "rawData/ansAcc.csv",
        "PerceivedDifficultyPath": "rawData/percDiff.csv",
        "PupilDataFilteredPath": "rawData/PupilData_cleaned.csv",
        "Questions": "rawData/questionInfo.csv"
        }
}

In [4]:
def read_json(path):
    with open(path, encoding='utf-8') as inFile:
        return json.load(inFile)
    
# Function to safely convert string to list or return None if input is not a string
def safe_eval(x):
    try:
        # Only try to evaluate strings that are not NaN (None in this context)
        if pd.notna(x):
            return ast.literal_eval(x)
    except ValueError:
        return None

In [5]:
#Load all the data
for exp_key, exp_info in Experiments.items():
    
    print(f"Loading data for {exp_key}")
    
    # Update the dictionary with the new attributes
    
    exp_info['QuestionsDf'] = pd.read_csv(exp_info['Questions'])
    exp_info['QuestionsDf']['Experiment'] = exp_key
    # Convert the 'Data' column from string to actual list, handling NaN safely
    exp_info['QuestionsDf']['Relevant elements ids'] = exp_info['QuestionsDf']['Relevant elements ids'].apply(safe_eval)
    
    exp_info['ETDataDf'] = pd.read_csv(exp_info['ETDataPath'])
    exp_info['ETDataDf']['Experiment'] = exp_key
    
    exp_info['AnswerDataDf'] = pd.read_csv(exp_info['AnswerDataPath'])
    exp_info['AnswerDataDf']['Experiment'] = exp_key
    
    exp_info['PupilDataFilteredDf'] = pd.read_csv(exp_info['PupilDataFilteredPath'])
    exp_info['PupilDataFilteredDf']['Experiment'] = exp_key
    
    exp_info['PerceivedDifficultyDf'] = pd.read_csv(exp_info['PerceivedDifficultyPath'])
    exp_info['PerceivedDifficultyDf']['Experiment'] = exp_key

Loading data for Exp1


In [6]:
#Adding questions info to the different datasets
for exp_key, exp_info in Experiments.items():
    exp_info['ETDataDf'] = pd.merge(exp_info['ETDataDf'], exp_info['QuestionsDf'], left_on=['currentQuestion', 'Experiment'], right_on=['id', 'Experiment'], how='left')
    exp_info['AnswerDataDf'] = pd.merge(exp_info['AnswerDataDf'], exp_info['QuestionsDf'], left_on=['questionID', 'Experiment'], right_on=['id', 'Experiment'], how='left')
    exp_info['PerceivedDifficultyDf'] = pd.merge(exp_info['PerceivedDifficultyDf'], exp_info['QuestionsDf'], left_on=['questionID', 'Experiment'], right_on=['id', 'Experiment'], how='left')
    exp_info['PupilDataFilteredDf'] = pd.merge(exp_info['PupilDataFilteredDf'], exp_info['QuestionsDf'], left_on=['currentQuestion', 'Experiment'], right_on=['id', 'Experiment'], how='left')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1112: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [7]:
###############################
####
#### Computing measures
####
###############################

In [8]:
####################
## Run count (visual assocations -> cognitive integration)
#####################

In [9]:
# Filter Transitions
# Function to check if at least one element of a transition tuple is in the relevant elements list
def filter_transitions(row):
    return any(item in row['Relevant elements ids'] for item in row['Transition'])

# Compute the runCount measures
def ComputeRunCount(eTDataDf):

    #Keep fixation data only
    fixationData = eTDataDf.loc[(~eTDataDf['FixID'].isna())].copy(deep=True)

    """
     2 rows are removed for participant SP11. This concerns the questions local and global (control-flow) Exclusiveness, since the participant 
     skipped the answer for the local Exclusiveness question by mistake. Hence the duration and accuracy were biased!
    """
    fixationData = fixationData.drop(fixationData[(fixationData['participant'] == 'SP11-no') & (fixationData['Type3'] == 'Exclusiveness')].index)

    # Filter to keep only process model AOIs
    pattern = r'^file-explorer-file_|Process'
    fixationData = fixationData[~fixationData['element'].str.match(pattern,na=False)]

    #converstion
    fixationData['Relevant elements ids'] = fixationData['Relevant elements ids'].apply(
        lambda x: tuple(x) if isinstance(x, list) else ()
    )

    #merge by consecutive element to obtain visits to elements (dwells)
    fixationData["element_"] = fixationData["element"]
    dwellData = fixationData.groupby([(fixationData['element'].shift() != fixationData['element']).cumsum(),'element_','participant','currentQuestion','Type1','Type2','id','Relevant elements ids'], as_index=False).agg(fixations=('FixID','count'))


    # Convert 'Relevant elements ids' from tuples back to lists for further processing
    dwellData['Relevant elements ids'] = dwellData['Relevant elements ids'].apply(list)

    # Identify Transitions
    # Shift the 'tabName_' within each 'currentQuestion' to create a column for the next visit
    dwellData['Next_element'] = dwellData.groupby('currentQuestion')['element_'].shift(-1)

    # Create a new DataFrame with transitions
    transitionsDf = dwellData.dropna(subset=['Next_element'])  # Drop rows where Next_TabName is NaN (last rows in each group)
    transitionsDf = transitionsDf.copy(deep=True)
    transitionsDf['Transition'] = list(zip(transitionsDf['element_'], transitionsDf['Next_element']))

    # Apply filter
    filteredTransitions = transitionsDf[transitionsDf.apply(filter_transitions, axis=1)]

    # Keep only relevant columns
    filteredTransitions = filteredTransitions[['participant', 'currentQuestion', 'Type1','Type2', 'Transition']]

    #Calculate runCount per task
    runCountPerTask = filteredTransitions.groupby(['participant','currentQuestion','Type1','Type2'],as_index=False).agg(runCount=('Transition','count'))
    
    
    return runCountPerTask

In [10]:
ComputeRunCount(Experiments['Exp1']['ETDataDf'])

,participant,currentQuestion,Type1,Type2,runCount
0,KP1-no,7.0,Local,Flow-based,4
1,KP1-no,10.0,Local,Flow-based,9
2,KP1-no,13.0,Local,Flow-based,15
3,KP1-no,16.0,Local,Flow-based,4
4,KP1-no,19.0,Global,Flow-based,6
...,...,...,...,...,...
606,SP9-no,34.0,Local,Circumstantial,32
607,SP9-no,37.0,Local,Circumstantial,17
608,SP9-no,40.0,Global,Circumstantial,17
609,SP9-no,43.0,Global,Circumstantial,17


In [11]:
####################
## Comprehension Efficiency
#####################

In [12]:
def ComputeComprehensionEfficiency(eTDataDf,asnwersDataDf):

    # Keep only main tasks
    eTDataDf = eTDataDf[eTDataDf['id'].notna()].copy(deep=True)

    """
    2 rows are removed for participant SP11. This concerns the questions local and global (control-flow) Exclusiveness, since the participant 
    skipped the answer for the local Exclusiveness question by mistake. Hence the duration and accuracy were biased!
    """
    eTDataDf = eTDataDf.drop(eTDataDf[(eTDataDf['participant'] == 'SP11-no') & (eTDataDf['Type3'] == 'Exclusiveness')].index)

    # Group data
    eTDataDfGrouped = eTDataDf.groupby(['participant','currentQuestion','Type1','Type2','Type3'], as_index=False)

    # Perform aggregations on grouping:
    eTDataDfAggregated= eTDataDfGrouped.agg(
      TaskDuration=('Timestamp', (lambda x: x.iloc[-1]-x.iloc[0])),

    )
    #flatten
    eTDataDfAggregated = eTDataDfAggregated.reset_index()

    #Merge with answers data
    ETandAnswersData = pd.merge(eTDataDfAggregated, asnwersDataDf, left_on=['participant', 'currentQuestion','Type1','Type2','Type3'], right_on=['participant', 'questionID','Type1','Type2','Type3'])

    #convert task duration from ms to s
    ETandAnswersData["TaskDuration"] = ETandAnswersData["TaskDuration"]/1000

    #Compute comprehension effciency
    ETandAnswersData["comprehensionEfficiency"] = ETandAnswersData["AnswerAccuracy"]/ETandAnswersData["TaskDuration"]

    return ETandAnswersData[['participant', 'currentQuestion','Type1','Type2','comprehensionEfficiency']]

In [13]:
ComputeComprehensionEfficiency(Experiments['Exp1']['ETDataDf'],Experiments['Exp1']['AnswerDataDf'])

,participant,currentQuestion,Type1,Type2,comprehensionEfficiency
0,KP1-no,7.0,Local,Flow-based,0.013104
1,KP1-no,10.0,Local,Flow-based,0.017223
2,KP1-no,13.0,Local,Flow-based,0.009525
3,KP1-no,16.0,Local,Flow-based,0.053349
4,KP1-no,19.0,Global,Flow-based,0.042280
...,...,...,...,...,...
609,SP9-no,34.0,Local,Circumstantial,0.015482
610,SP9-no,37.0,Local,Circumstantial,0.009671
611,SP9-no,40.0,Global,Circumstantial,0.007109
612,SP9-no,43.0,Global,Circumstantial,0.010269


In [14]:
####################
## Perceived difficulty
#####################

In [15]:
def ComputePerceivedDifficulty(PerceivedDiffDf):
    """
    2 rows are removed for participant SP11. This concerns the questions local and global (control-flow) Exclusiveness, since the participant 
    skipped the answer for the local Exclusiveness question by mistake. Hence the duration and accuracy were biased!
    """

    # Filtering out the specific conditions
    # Ensuring the participant and type conditions are filtered within the same DataFrame
    mask = (PerceivedDiffDf['participant'] == 'SP11-no') & (PerceivedDiffDf['Type3'] == 'Exclusiveness')
    PerceivedDiffDf = PerceivedDiffDf[~mask]

    PerceivedDiffDf = PerceivedDiffDf.copy(deep=True)
    PerceivedDiffDf['currentQuestion'] = PerceivedDiffDf['questionID']

    # Mean and std perceived difficulty in the whole dataset
    print("Info: mean perceived difficulty in the whole dataset", PerceivedDiffDf['difficultyScore'].mean())

    return PerceivedDiffDf[['participant', 'currentQuestion', 'Type1', 'Type2', 'difficultyScore']]


In [16]:
ComputePerceivedDifficulty(Experiments['Exp1']['PerceivedDifficultyDf'])

Info: mean perceived difficulty in the whole dataset 1.2834890965732086


,participant,currentQuestion,Type1,Type2,difficultyScore
0,SP10-no,28,Global,Flow-based,0
1,SP2-no,28,Global,Flow-based,3
2,KP7-no,28,Global,Flow-based,2
3,SP14-no,28,Global,Flow-based,0
4,SP20-no,28,Global,Flow-based,2
...,...,...,...,...,...
639,SP1-no,37,Local,Circumstantial,1
640,KP4-no,37,Local,Circumstantial,1
641,KP19-no,37,Local,Circumstantial,2
642,SP17-no,37,Local,Circumstantial,1


In [17]:
####################
## Mental Processing Fixations
#####################

In [18]:
def ComputeMentalProcessingFixations(eTDataDf):

    #drop N/A
    fixationData = eTDataDf.loc[(~eTDataDf['FixID'].isna()) & (~eTDataDf['currentQuestion'].isna())].copy(deep=True)

    """
    2 rows are removed for participant SP11. This concerns the questions local and global (control-flow) Exclusiveness, since the participant 
    skipped the answer for the local Exclusiveness question by mistake. Hence the duration and accuracy were biased!
    """
    fixationData = fixationData.drop(fixationData[(fixationData['participant'] == 'SP11-no') & (fixationData['Type3'] == 'Exclusiveness')].index)

    #set fixation threshold
    fixationData = fixationData[(fixationData['Fixation Duration']>250)]

    #Compute Fixation Count
    mentalProcessingFixations = fixationData.groupby(['participant','currentQuestion','Type1','Type2'], as_index=False).agg(MentalProcessingFixationsCount=('FixID','count'))


    return mentalProcessingFixations

In [19]:
ComputeMentalProcessingFixations(Experiments['Exp1']['ETDataDf'])

,participant,currentQuestion,Type1,Type2,MentalProcessingFixationsCount
0,KP1-no,7.0,Local,Flow-based,32
1,KP1-no,10.0,Local,Flow-based,35
2,KP1-no,13.0,Local,Flow-based,38
3,KP1-no,16.0,Local,Flow-based,5
4,KP1-no,19.0,Global,Flow-based,8
...,...,...,...,...,...
608,SP9-no,34.0,Local,Circumstantial,71
609,SP9-no,37.0,Local,Circumstantial,98
610,SP9-no,40.0,Global,Circumstantial,148
611,SP9-no,43.0,Global,Circumstantial,80


In [20]:
####################
## Low/High Index of Pupil Activity (LHIPA)
#####################

In [21]:
# LHIPA is expected to decrease with increased cognitive load
def lhipa(dx):
    
    
    # find max decomposition level 
    d = list(dx['pupilSize'])

    if len(d) > 100:
        d.insert(0, 0)
        sym = 'sym5'  # modified sym5
        w = pywt.Wavelet(sym)
        maxlevel = pywt.dwt_max_level(len(d), filter_len=w.dec_len)
        hif, lof = 1, int(maxlevel / 2)

        # get detail coefficients of pupil diameter signal d
        cD_H = pywt.downcoef('d', d, sym, 'per', level=hif)
        cD_L = pywt.downcoef('d', d, sym, 'per', level=lof)

        # normalize by 1/ 2j􀀀
        cD_H = [x / math.sqrt(2 ** hif) for x in cD_H]
        cD_L = [x / math.sqrt(2 ** lof) for x in cD_L]

        # obtain the LH:HF ratio
        cD_LH = cD_L

        for i in range(len(cD_L)):
            cD_LH[i] = cD_L[i] / cD_H[int((2 ** lof) / (2 ** hif) * i)]

        # detect modulus maxima , see Duchowski et al. [15]
        cD_LHm = modmax(cD_LH)

        # threshold using universal threshold luniv􀀀= sˆ􀀀 (2logn)
        # where sˆ􀀀 is the standard deviation of the noise
        luniv = np.std(cD_LHm) * math.sqrt(2.0 * np.log2(len(cD_LHm)))
        cD_LHt = pywt.threshold(cD_LHm, luniv, mode="less")

        # get signal duration (in seconds)
        d2 = list(dx['Timestamp'])
        tt = (d2[-1] - d2[0]) / 1000

        # compute LHIPA
        ctr = 0
        for i in range(len(cD_LHt)):
            if math.fabs(cD_LHt[i]) > 0:
                ctr += 1
        LHIPA = float(ctr) / tt
        return LHIPA
    return np.nan

def modmax(d):
    # compute signal
    m = [math.fabs(di) for di in d]

    # if value is larger than both neighbours, and strictly # larger than either, then it is a local maximum
    t = [0.0] * len(d)
    for i in range(len(d)):
        ll = m[i - 1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i + 1] if i < len(d) - 1 else m[i]  # change len(d)-2 to len(d)-1
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
            # compute magnitude
            t[i] = math.sqrt(d[i] ** 2)
        else:
            t[i] = 0.0
    return t

def ComputeLHIPA(pupilDataDf):

    data = pupilDataDf[(~pupilDataDf['tabName'].isna())].copy(deep=True)

    """
    2 rows are removed for participant SP11. This concerns the questions local and global (control-flow) Exclusiveness, since the participant 
    skipped the answer for the local Exclusiveness question by mistake. Hence the duration and accuracy were biased!
    """
    data = data.drop(data[(data['participant'] == 'SP11-no') & (data['Type3'] == 'Exclusiveness')].index)

    # group by 'participant', 'currentQuestion', 'Type1', 'Type2'
    data = data.groupby(['participant', 'currentQuestion', 'Type1', 'Type2'])

    # Compute LHIPA for each group
    data = data.apply(lhipa).reset_index()
    data = data.rename(columns={0: 'LHIPA'})

    #remove extremely low values (close to 0) and nans
    data = data[data["LHIPA"]>0.4]
    data.dropna(subset=["LHIPA"], inplace=True)

    return data

In [22]:
ComputeLHIPA(Experiments['Exp1']['PupilDataFilteredDf'])

,participant,currentQuestion,Type1,Type2,LHIPA
0,KP1-no,7.0,Local,Flow-based,0.707228
1,KP1-no,10.0,Local,Flow-based,1.326169
2,KP1-no,13.0,Local,Flow-based,0.666781
3,KP1-no,16.0,Local,Flow-based,1.170042
4,KP1-no,19.0,Global,Flow-based,1.436487
...,...,...,...,...,...
607,SP9-no,34.0,Local,Circumstantial,0.846246
608,SP9-no,37.0,Local,Circumstantial,0.753997
609,SP9-no,40.0,Global,Circumstantial,0.767401
610,SP9-no,43.0,Global,Circumstantial,0.739028


In [23]:
###############################
#### Compile one dataset with all measures
###############################

In [24]:
# Function to load each DataFrame
def load_data():
    df_lhipa = ComputeLHIPA(Experiments['Exp1']['PupilDataFilteredDf'])
    df_mental = ComputeMentalProcessingFixations(Experiments['Exp1']['ETDataDf'])
    df_difficulty = ComputePerceivedDifficulty(Experiments['Exp1']['PerceivedDifficultyDf'])
    df_efficiency = ComputeComprehensionEfficiency(Experiments['Exp1']['ETDataDf'], Experiments['Exp1']['AnswerDataDf'])
    df_run_count = ComputeRunCount(Experiments['Exp1']['ETDataDf'])
    return [df_lhipa, df_mental, df_difficulty, df_efficiency, df_run_count]

# Function to merge all DataFrames
def merge_dataframes(dfs):
    return reduce(lambda left, right: pd.merge(left, right, on=['participant', 'currentQuestion', 'Type1', 'Type2'], how='outer'), dfs)

# Load DataFrames
dataframes = load_data()

# Merge all DataFrames
allMeasures = merge_dataframes(dataframes)

# Display the head of the merged DataFrame
allMeasures.head()

Info: mean perceived difficulty in the whole dataset 1.2834890965732086


,participant,currentQuestion,Type1,Type2,LHIPA,MentalProcessingFixationsCount,difficultyScore,comprehensionEfficiency,runCount
0,KP1-no,7.0,Local,Flow-based,0.707228,32.0,1,0.013104,4.0
1,KP1-no,10.0,Local,Flow-based,1.326169,35.0,1,0.017223,9.0
2,KP1-no,13.0,Local,Flow-based,0.666781,38.0,2,0.009525,15.0
3,KP1-no,16.0,Local,Flow-based,1.170042,5.0,0,0.053349,4.0
4,KP1-no,19.0,Global,Flow-based,1.436487,8.0,0,0.042280,6.0


In [25]:
###############################
####
#### Testing the effect of Fragmentation (Type1 (Local/Global)) on runCount considering the moderating effect of Task perspective (Type2 (Flow-based/Circumenstantial))
####
###############################

In [26]:
###############################
#### Descriptive statistics
###############################

In [27]:
def compute_grouped_mean(df, firstGrouping, secondGrouping, measure_col,additional_filters=None):
     
    # Apply additional filters if provided
    if additional_filters:
        for key, value in additional_filters.items():
            df = df[df[key] == value]

    # To get one data point per factor level: apply firstGrouping then calculate the mean for each measure 
    grouped_means = df.groupby(firstGrouping)[[measure_col]].mean().reset_index()
    
    # regroup by secondGrouping and compute the mean
    overall_mean = grouped_means.groupby(secondGrouping,as_index=False)[[measure_col]].mean()

    return overall_mean

In [28]:
measure = 'runCount'
means = compute_grouped_mean(df=allMeasures,
                             firstGrouping=['Type1', 'Type2', 'participant'],
                             secondGrouping=['Type1', 'Type2'],
                             measure_col=measure,
                             additional_filters={})

# Pivot the table
pivot_df = means.pivot_table(index='Type2', columns='Type1', values=measure)

# Reorder the rows to have 'Flow-based' first
pivot_df = pivot_df.reindex(['Flow-based', 'Circumstantial'])

# Reorder the columns to have 'Local' first and then 'Global'
pivot_df = pivot_df[['Local', 'Global']]

# Set the display precision to 3 decimal places
pd.options.display.float_format = '{:,.3f}'.format

# Display the pivoted data
display(pivot_df)

Type1,Local,Global
Type2,,
Flow-based,16.159,31.578
Circumstantial,20.962,20.379


In [29]:
###############################
#### Testing for Moderating effect using MEMORE in SPSS
###############################

In [30]:
######
## 1. Preparing data to be processed by SPSS
#######

In [31]:
allMeasures

,participant,currentQuestion,Type1,Type2,LHIPA,MentalProcessingFixationsCount,difficultyScore,comprehensionEfficiency,runCount
0,KP1-no,7.000,Local,Flow-based,0.707,32.000,1,0.013,4.000
1,KP1-no,10.000,Local,Flow-based,1.326,35.000,1,0.017,9.000
2,KP1-no,13.000,Local,Flow-based,0.667,38.000,2,0.010,15.000
3,KP1-no,16.000,Local,Flow-based,1.170,5.000,0,0.053,4.000
4,KP1-no,19.000,Global,Flow-based,1.436,8.000,0,0.042,6.000
...,...,...,...,...,...,...,...,...,...
637,SP21-no,7.000,Local,Flow-based,nan,nan,1,nan,nan
638,KP16-no,7.000,Local,Flow-based,nan,nan,0,0.140,nan
639,SP22-no,7.000,Local,Flow-based,nan,nan,2,nan,nan
640,SP21-no,37.000,Local,Circumstantial,nan,nan,2,nan,nan


In [32]:
#For the data to be processed by SPSS the following formatting is needed
"""

- Transform Type1:

Set the value to 0 if the entry is "Local".
Set the value to 1 if the entry is "Global".

- Transform Type2:

Set the value to 0 if the entry is "Flow-based".
Set the value to 1 if the entry is "Circumstantial".

- Rename Columns:

Shorten the names of all measures to be meaningful yet concise, ideally less than 8 characters."""

def applyDataFormattingForSPSS(data):
    
    data = data.copy(deep=True)

    # Transform 'Type1' and 'Type2' according to specified rules
    data['Type1'] = data['Type1'].map({'Local': 0, 'Global': 1})
    data['Type2'] = data['Type2'].map({'Flow-based': 0, 'Circumstantial': 1})

    # Rename columns to be shorter but meaningful
    new_column_names = {
        'currentQuestion': 'currQ',  # Shortened 'currentQuestion' to 'currQ'
        'Type1': 'Type1',  # Kept 'Type1' as it is under 8 characters
        'Type2': 'Type2',  # Kept 'Type2' as it is under 8 characters
        'LHIPA': 'LHIPA',  # Kept 'LHIPA' as is, assuming it's a specific measure name
        'MentalProcessingFixationsCount': 'MPFCnt',  # Shortened to 'MPFCnt'
        'difficultyScore': 'diffScr',  # Shortened to 'diffScr'
        'comprehensionEfficiency': 'compEff',  # Shortened to 'compEff'
        'runCount': 'runCnt'  # Shortened 'runCount' to 'runCnt'
    }

    # Apply the new column names to the DataFrame
    data.rename(columns=new_column_names, inplace=True)

    return data

In [33]:
applyDataFormattingForSPSS(allMeasures)

,participant,currQ,Type1,Type2,LHIPA,MPFCnt,diffScr,compEff,runCnt
0,KP1-no,7.000,0,0,0.707,32.000,1,0.013,4.000
1,KP1-no,10.000,0,0,1.326,35.000,1,0.017,9.000
2,KP1-no,13.000,0,0,0.667,38.000,2,0.010,15.000
3,KP1-no,16.000,0,0,1.170,5.000,0,0.053,4.000
4,KP1-no,19.000,1,0,1.436,8.000,0,0.042,6.000
...,...,...,...,...,...,...,...,...,...
637,SP21-no,7.000,0,0,nan,nan,1,nan,nan
638,KP16-no,7.000,0,0,nan,nan,0,0.140,nan
639,SP22-no,7.000,0,0,nan,nan,2,nan,nan
640,SP21-no,37.000,0,1,nan,nan,2,nan,nan


In [34]:
## To run MEMORE, you should have one data point per condition and the data should be organized in pairwise manner

In [35]:
def formatForMEMORE(data):
    
    data = data.copy(deep=True)
    
    # To get one data point per factor level: apply firstGrouping then calculate the mean for each measure 
    grouped_means = data.groupby(['Type1','Type2','participant'])[['LHIPA', 'MPFCnt', 'diffScr',
       'compEff', 'runCnt']].mean().reset_index()
    
    #Split into local, global
    localTasksData = grouped_means.loc[(grouped_means['Type1']==0)].dropna()
    globalTasksData = grouped_means.loc[(grouped_means['Type1']==1)].dropna()
    
    #merge to allow pairwise comparision between local and global. Add suffix _L and _G depending on wether the measures corresponds to a local or global task
    merged = localTasksData.merge(globalTasksData, on=['participant','Type2'], suffixes=('_L', '_G'), how='inner')
    
    # Define the columns to keep and their new order
    columns_to_keep = [
    'participant', 'Type2', 
    'LHIPA_L', 'MPFCnt_L', 'diffScr_L', 'compEff_L', 'runCnt_L',
    'LHIPA_G', 'MPFCnt_G', 'diffScr_G', 'compEff_G', 'runCnt_G'
    ]

    # Select and reorder the columns in the DataFrame
    merged = merged[columns_to_keep]


    return merged

In [36]:
formatForMEMORE(applyDataFormattingForSPSS(allMeasures))

,participant,Type2,LHIPA_L,MPFCnt_L,diffScr_L,compEff_L,runCnt_L,LHIPA_G,MPFCnt_G,diffScr_G,compEff_G,runCnt_G
0,KP1-no,0,0.968,27.500,1.000,0.023,8.000,0.999,22.750,1.000,0.021,8.250
1,KP10-no,0,1.517,34.500,0.750,0.028,17.250,0.711,128.500,3.000,0.008,53.000
2,KP11-no,0,1.084,78.750,1.250,0.013,13.250,1.108,37.000,2.750,0.005,15.750
3,KP12-no,0,1.565,10.500,0.000,0.033,6.250,1.130,53.000,1.250,0.013,23.000
4,KP13-no,0,1.044,38.500,0.750,0.017,23.750,0.863,97.500,3.000,0.004,48.750
...,...,...,...,...,...,...,...,...,...,...,...,...
79,SP5-no,1,1.108,73.333,2.000,0.006,22.667,0.891,74.333,2.333,0.013,14.333
80,SP6-no,1,1.370,23.333,1.000,0.022,17.000,0.945,58.333,1.333,0.014,24.000
81,SP7-no,1,0.724,82.667,0.333,0.009,46.000,0.715,60.000,1.333,0.011,35.333
82,SP8-no,1,1.232,37.667,2.333,0.013,14.667,0.716,73.333,1.000,0.012,19.000


In [37]:
# To run moderator analysis in MEMORE, runCount should be normally distributed

In [38]:
def normalityAdjc(data):
    """
    Tests for normality on 'runCnt_L' and 'runCnt_G' in the provided DataFrame,
    applies log transformation if they are not normally distributed,
    and re-tests for normality. 
    """
    
    data = data.copy(deep=True)

    # Initial test for normal distribution
    shapiro_rcL = shapiro(data['runCnt_L'].dropna())
    shapiro_rcG = shapiro(data['runCnt_G'].dropna())
    
    print("Initial Shapiro-Wilk Test Results:")
    print("runCnt_L:", shapiro_rcL)
    print("runCnt_G:", shapiro_rcG)

    # Apply log transformation if not normally distributed (p < 0.05 indicates non-normal)
    if shapiro_rcL.pvalue < 0.05:
        data['runCnt_L'] = np.log(data['runCnt_L'].replace(0, np.nan).dropna() + 1)
        shapiro_rcL = shapiro(data['runCnt_L'])
        print("After Log Transformation Shapiro-Wilk Test Result for runCnt_L:", shapiro_rcL)
    
    if shapiro_rcG.pvalue < 0.05:
        data['runCnt_G'] = np.log(data['runCnt_G'].replace(0, np.nan).dropna() + 1)
        shapiro_rcG = shapiro(data['runCnt_G'])
        print("After Log Transformation Shapiro-Wilk Test Result for runCnt_G:", shapiro_rcG)

    return data

In [39]:
normalityAdjc(formatForMEMORE(applyDataFormattingForSPSS(allMeasures)))

Initial Shapiro-Wilk Test Results:
runCnt_L: ShapiroResult(statistic=0.9194626808166504, pvalue=6.032393503119238e-05)
runCnt_G: ShapiroResult(statistic=0.8877193331718445, pvalue=2.3827296899980865e-06)
After Log Transformation Shapiro-Wilk Test Result for runCnt_L: ShapiroResult(statistic=0.9822677969932556, pvalue=0.30157795548439026)
After Log Transformation Shapiro-Wilk Test Result for runCnt_G: ShapiroResult(statistic=0.9847720861434937, pvalue=0.42647406458854675)


,participant,Type2,LHIPA_L,MPFCnt_L,diffScr_L,compEff_L,runCnt_L,LHIPA_G,MPFCnt_G,diffScr_G,compEff_G,runCnt_G
0,KP1-no,0,0.968,27.500,1.000,0.023,2.197,0.999,22.750,1.000,0.021,2.225
1,KP10-no,0,1.517,34.500,0.750,0.028,2.904,0.711,128.500,3.000,0.008,3.989
2,KP11-no,0,1.084,78.750,1.250,0.013,2.657,1.108,37.000,2.750,0.005,2.818
3,KP12-no,0,1.565,10.500,0.000,0.033,1.981,1.130,53.000,1.250,0.013,3.178
4,KP13-no,0,1.044,38.500,0.750,0.017,3.209,0.863,97.500,3.000,0.004,3.907
...,...,...,...,...,...,...,...,...,...,...,...,...
79,SP5-no,1,1.108,73.333,2.000,0.006,3.164,0.891,74.333,2.333,0.013,2.730
80,SP6-no,1,1.370,23.333,1.000,0.022,2.890,0.945,58.333,1.333,0.014,3.219
81,SP7-no,1,0.724,82.667,0.333,0.009,3.850,0.715,60.000,1.333,0.011,3.593
82,SP8-no,1,1.232,37.667,2.333,0.013,2.752,0.716,73.333,1.000,0.012,2.996


In [40]:
#Expert data for analysis in SPSS using MEMORE
normalityAdjc(formatForMEMORE(applyDataFormattingForSPSS(allMeasures))).to_csv("dataforSPSS.csv")

Initial Shapiro-Wilk Test Results:
runCnt_L: ShapiroResult(statistic=0.9194626808166504, pvalue=6.032393503119238e-05)
runCnt_G: ShapiroResult(statistic=0.8877193331718445, pvalue=2.3827296899980865e-06)
After Log Transformation Shapiro-Wilk Test Result for runCnt_L: ShapiroResult(statistic=0.9822677969932556, pvalue=0.30157795548439026)
After Log Transformation Shapiro-Wilk Test Result for runCnt_G: ShapiroResult(statistic=0.9847720861434937, pvalue=0.42647406458854675)


In [41]:
"""
In SPSS run the MEMORE macro (https://www.akmontoya.com/spss-and-sas-macros)
Then, run the following command: MEMORE Y=runCnt_G runCnt_L/W=Type2 /model=2

SPSS output:

Conditional Effect of 'X' on Y at values of moderator(s) 

      Type2     Effect         SE          t          p       LLCI       ULCI 
      .0000      .5965      .0538    11.0922      .0000      .4895      .7035 
     1.0000      .0215      .0538      .4006      .6897     -.0854      .1285

X is Type1 (Local/Global)
Y is RunCount (runCnt_L/runCnt_G)
Type2 is TaskPerspective (0=Flow-based, 1=Circumstantial)

- Rule to check for statistical significance: p<0.05 suggests that the conditional effect is signiciant
- Finding: The conditional effect is present in Type2=0 (i.e., flow-based tasks) but not in (Type2=1 i.e., circumstantial tasks)
- Conclusion: this confirms that Type2 (Task Perspective) is a moderator variable
"""

"\nIn SPSS run the MEMORE macro (https://www.akmontoya.com/spss-and-sas-macros)\nThen, run the following command: MEMORE Y=runCnt_G runCnt_L/W=Type2 /model=2\n\nSPSS output:\n\nConditional Effect of 'X' on Y at values of moderator(s) \n\n      Type2     Effect         SE          t          p       LLCI       ULCI \n      .0000      .5965      .0538    11.0922      .0000      .4895      .7035 \n     1.0000      .0215      .0538      .4006      .6897     -.0854      .1285\n\nX is Type1 (Local/Global)\nY is RunCount (runCnt_L/runCnt_G)\nType2 is TaskPerspective (0=Flow-based, 1=Circumstantial)\n\n- Rule to check for statistical significance: p<0.05 suggests that the conditional effect is signiciant\n- Finding: The conditional effect is present in Type2=0 (i.e., flow-based tasks) but not in (Type2=1 i.e., circumstantial tasks)\n- Conclusion: this confirms that Type2 (Task Perspective) is a moderator variable\n"

In [42]:
###############################
#### Follow-up moderator investigation using Wilcoxon signed rank test
###############################

In [43]:
allMeasures

,participant,currentQuestion,Type1,Type2,LHIPA,MentalProcessingFixationsCount,difficultyScore,comprehensionEfficiency,runCount
0,KP1-no,7.000,Local,Flow-based,0.707,32.000,1,0.013,4.000
1,KP1-no,10.000,Local,Flow-based,1.326,35.000,1,0.017,9.000
2,KP1-no,13.000,Local,Flow-based,0.667,38.000,2,0.010,15.000
3,KP1-no,16.000,Local,Flow-based,1.170,5.000,0,0.053,4.000
4,KP1-no,19.000,Global,Flow-based,1.436,8.000,0,0.042,6.000
...,...,...,...,...,...,...,...,...,...
637,SP21-no,7.000,Local,Flow-based,nan,nan,1,nan,nan
638,KP16-no,7.000,Local,Flow-based,nan,nan,0,0.140,nan
639,SP22-no,7.000,Local,Flow-based,nan,nan,2,nan,nan
640,SP21-no,37.000,Local,Circumstantial,nan,nan,2,nan,nan


In [44]:
def wilcoxonAnalysis(data, grouping_cols, factor_level_col, factor_levels_sets, measure_col, tail, additional_filters=None):
    
    data = data.copy(deep=True)
    
    
    print(additional_filters)
    
    # Apply additional filters if provided
    if additional_filters:
        for key, value in additional_filters.items():
            data = data[data[key] == value]
            
            
    # To get one data point per factor level: group by group_cols and calculate the mean
    data = data.groupby(grouping_cols,as_index=False).agg({measure_col:'mean'})
    
    
    # for each pair of factors in factor_levels_sets to compare in a pairwise approach, generate the two datasets data_factor_level1 and data_factor_level2
    for factor_level1, factor_level2 in factor_levels_sets:
        
        print(factor_level1,' vs. ',factor_level2)
        
        data_factor_level1 = data.loc[(data[factor_level_col]==factor_level1)][grouping_cols+[measure_col]].dropna()
        data_factor_level2 = data.loc[(data[factor_level_col]==factor_level2)][grouping_cols+[measure_col]].dropna()
        
        #remove factor_level_col from grouping_cols
        grouping_cols.remove(factor_level_col)
        
        #merge data_factor_level1 and data_factor_level2 based on grouping_cols - factor_level_col 
        merged = data_factor_level1.merge(data_factor_level2, on=grouping_cols, suffixes=('_fl1', '_fl2'), how='inner')
        
        # Exclude pairs with zero difference
        merged = merged[merged[f'{measure_col}_fl1'] != merged[f'{measure_col}_fl2']]
        
        # Calculate Wilcoxon test
        stat, p = stats.wilcoxon(merged[f'{measure_col}_fl1'], merged[f'{measure_col}_fl2'], alternative=tail)
       
        # Display in green if p < 0.05, else red
        color = 'green' if p < 0.05 else 'red'
        display(HTML(f"<span style='color: {color};'>{factor_level1} vs {factor_level2}: p-value = {p}</span>"))

In [45]:
res = wilcoxonAnalysis(data= allMeasures, 
               grouping_cols=['participant','Type1','Type2'], 
               factor_level_col='Type1', 
               factor_levels_sets =[('Local', 'Global')], 
               measure_col = 'runCount',
               tail = 'less',
               additional_filters=
                                {
                                    'Type2':'Flow-based'
                                })

res = wilcoxonAnalysis(data= allMeasures, 
               grouping_cols=['participant','Type1','Type2'], 
               factor_level_col='Type1', 
               factor_levels_sets =[('Local', 'Global')], 
               measure_col = 'runCount',
               tail = 'less',
               additional_filters=
                                {
                                    'Type2':'Circumstantial'
                                })

{'Type2': 'Flow-based'}
Local  vs.  Global


{'Type2': 'Circumstantial'}
Local  vs.  Global


In [46]:
###############################
####
#### Testing the effect of Cognitive Integration (runCount) on Cognitive Load and Comprehension, for different groups: flow-based local, flow-based global, circumstantial local, circumstantial global  
####
###############################

In [47]:
def regression(data):
    
    data = data.copy(deep=True)
    
    measures = ['MentalProcessingFixationsCount', 'LHIPA', 'difficultyScore',
       'comprehensionEfficiency']
    indepVarName = 'runCount'
    
    # iterate over flow-based local, flow-based global, circumstantial local, circumstantial global
    for type2 in data['Type2'].unique():
        for type1 in data['Type1'].unique():
            print("-------------------------------------------------")
            print(f"Task Type: {type2} - {type1}:")
                
            for depVarName in measures:
                
                print("")
                print(f"--- Dep var. {depVarName} -----")
                
                # Keep relevant data matching type1 and type2
                filteredData = data[(data['Type1'] == type1) & (data['Type2'] == type2)]

                #Keep only relevant cols
                filteredData = filteredData[['participant', 'Type1', 'Type2',indepVarName, depVarName]]

                #dropNA
                filteredData = filteredData.dropna(subset=[indepVarName, depVarName])

                if filteredData.empty:
                    print(f"No data available for Group: {type1} - {type2}")
                    continue

                # Group by participant, type1, typ2 and compute the mean of the dependent variable to get one data point per condition
                grouped_df = filteredData.groupby(['participant', 'Type1', 'Type2'], as_index=False).agg(
                    {depVarName: 'mean', indepVarName: 'mean'}
                )

                formula = f"{depVarName} ~ {indepVarName}"

                model = smf.rlm(formula=formula, data=grouped_df,M=sm.robust.norms.Hampel())
                result = model.fit()

                summary = result.summary2().tables[1]  # Get the coefficients table
                summary['Coef.'] = summary['Coef.'].apply(lambda x: f"{x:.3f}" if abs(x) >= 0.001 else f"{x:.2e}")
                summary['Std.Err.'] = summary['Std.Err.'].apply(lambda x: f"{x:.3f}" if abs(x) >= 0.001 else f"{x:.2e}")
                summary['[0.025'] = summary['[0.025'].apply(lambda x: f"{x:.3f}" if abs(x) >= 0.001 else f"{x:.2e}")
                summary['0.975]'] = summary['0.975]'].apply(lambda x: f"{x:.3f}" if abs(x) >= 0.001 else f"{x:.2e}")

                display(summary)
                
                # Display summary in LaTeX
                #summaryLatex = summary.loc[summary.index != 'Intercept']
                #print(summaryLatex.to_latex(escape=False))

                # Print coefficients with full precision for indepVarName
                p = result.pvalues[indepVarName]

                # Display in green if p < 0.05, else red
                color = 'green' if p < 0.05 else 'red'
                display(HTML(f"<span style='color: {color};'>p-value = {p:.10e}</span>"))






In [48]:
regression(allMeasures)

-------------------------------------------------
Task Type: Flow-based - Local:

--- Dep var. MentalProcessingFixationsCount -----


,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Intercept,17.530,4.651,3.769,0.000,8.414,26.646
runCount,1.185,0.261,4.533,0.000,0.672,1.697



--- Dep var. LHIPA -----


,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Intercept,1.367,0.080,17.072,0.000,1.210,1.524
runCount,-0.009,0.004,-2.075,0.038,-0.018,-5.15e-04



--- Dep var. difficultyScore -----


...

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Intercept,1.073,0.115,9.323,0.000,0.847,1.299
runCount,-0.011,0.005,-2.052,0.040,-0.022,-4.96e-04



--- Dep var. difficultyScore -----


,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Intercept,1.802,0.413,4.362,0.000,0.992,2.612
runCount,-0.016,0.019,-0.818,0.414,-0.054,0.022



--- Dep var. comprehensionEfficiency -----


,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Intercept,0.016,0.002,7.959,0.000,0.012,0.020
runCount,-2.54e-04,9.28e-05,-2.734,0.006,-4.36e-04,-7.19e-05
